<a href="https://colab.research.google.com/github/aim56009/Bias_GAN/blob/master/data/run_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports 

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")
#%cd "/content/gdrive/MyDrive/data_gan"

Mounted at /content/gdrive


In [2]:
!git clone https://github.com/aim56009/Bias_GAN.git

Cloning into 'Bias_GAN'...
remote: Enumerating objects: 259, done.
remote: Counting objects: 100% (160/160), done.
remote: Compressing objects: 100% (150/150), done.
remote: Total 259 (delta 93), reused 24 (delta 7), pack-reused 99
Receiving objects: 100% (259/259), 79.65 MiB | 23.91 MiB/s, done.
Resolving deltas: 100% (125/125), done.


In [3]:
%%capture
!pip install pytorch_lightning

In [4]:
!pip install importlib-metadata==4.0.1
!pip install xarray==0.18.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 5.2.0
    Uninstalling importlib-metadata-5.2.0:
      Successfully uninstalled importlib-metadata-5.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.4.1 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 4.0.1 which is incompatible.
gym 0.25.2 requires importlib-metadata>=4.8.0; python_version < "3.10", but you have importlib-metadata 4.0.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.3/807.3 KB 12.4 MB/s eta 0:00:00
  Attempting uninstall: xarray
    Found existing installation: xarray 2022.12

In [5]:
from argparse import ArgumentParser
import warnings
warnings.filterwarnings('ignore')
from dataclasses import dataclass, field
from typing import List
import getpass

#from Bias_GAN.code.src.trainer import train_cycle_gan

# Training

In [6]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
import json

from Bias_GAN.code.src.model import CycleGAN
from Bias_GAN.code.src.data import DataModule
from Bias_GAN.code.src.utils import get_version, set_environment, get_checkpoint_path, save_config
from Bias_GAN.code.src.callbacks import get_cycle_gan_callbacks

In [11]:
def train_cycle_gan(config):
    """ Main routing to train the Cycle GAN """
    
    config = Config()
    version = get_version()
    print(f'Running model: {version}')
    checkpoint_path = get_checkpoint_path(config, version)
    set_environment()
    print("checkpoint_path before:",checkpoint_path) 
    tb_logger = TensorBoardLogger(config.tensorboard_path,
                           name=config.model_name,
                           default_hp_metric=False,
                           version = checkpoint_path)
                           #version = version

    trainer = pl.Trainer(gpus = 1,
                         max_epochs = config.epochs,
                         precision = 16, 
                         #progress_bar_refresh_rate = config.progress_bar_refresh_rate,
                         callbacks = get_cycle_gan_callbacks(checkpoint_path),
                         num_sanity_val_steps = 1,
                         logger = tb_logger,
                         log_every_n_steps = config.log_every_n_steps,
                         deterministic = False,
                         accelerator=accelerator) 

    datamodule = DataModule(config, training_batch_size = config.train_batch_size,
                                    test_batch_size = config.test_batch_size)

    datamodule.setup("fit")

    model = CycleGAN(epoch_decay = config.epochs // 2,
                     running_bias=config.running_bias)
    
    #model = MyLightningModule.load_from_checkpoint("/path/to/checkpoint.ckpt")

    trainer.fit(model, datamodule)

    print('Training finished')
    return model

# Config

In [12]:
@dataclass
class Config:
    """ 
    Training configuration parameters. For model evaluation parameters see
    src/configuration.py.
    """
    
    scratch_path: str = '/content/gdrive/MyDrive/bias_gan/results'
    tensorboard_path: str = f'{scratch_path}/'
    checkpoint_path: str = f'{scratch_path}/'
    config_path: str = f'{scratch_path}/'
    poem_path: str = f"/content/gdrive/MyDrive/bias_gan/data_gan/pr_gfdl-esm4_historical_regionbox_1979-2014.nc"
    era5_path: str = f"/content/gdrive/MyDrive/bias_gan/data_gan/pr_W5E5v2.0_regionbox_era5_1979-2014.nc"
   

    results_path: str = f'{scratch_path}/'
    projection_path: str = None

    train_start: int = 1979
    train_end: int = 1980 # set to 2000 for full run
    valid_start: int = 2004 #was 2001
    valid_end: int = 2004
    test_start: int = 2004
    test_end: int = 2014
    
    model_name: str = 'tibet_gan'

    epochs: int = 2 # set to 250 for reproduction
    progress_bar_refresh_rate: int = 1
    train_batch_size: int = 1
    test_batch_size: int = 64
    transforms: List = field(default_factory=lambda: ['log', 'normalize_minus1_to_plus1'])
    rescale: bool = False
    epsilon: float = 0.0001
    lazy: bool = False
    log_every_n_steps: int = 2 ### was 10
    norm_output: bool = True
    running_bias: bool = False


def main():
    _ = train_cycle_gan(Config())

# Run

In [13]:
accelerator="gpu"
main()

INFO:lightning_lite.utilities.seed:Global seed set to 42
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:Using bfloat16 Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Running model: 2023_01_05/10h_09m_09s_ID_ff20ff92-8ce0-11ed-bdb5-0242ac1c000c
checkpoint_path before: /content/gdrive/MyDrive/bias_gan/results//2023_01_05_10h_09m_09s


INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type          | Params
----------------------------------------
0 | d_A   | Discriminator | 2.8 M 
1 | d_B   | Discriminator | 2.8 M 
2 | g_A2B | Generator     | 449 K 
3 | g_B2A | Generator     | 449 K 
----------------------------------------
6.4 M     Trainable params
0         Non-trainable params
6.4 M     Total params
25.697    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training finished


In [ ]:
from datetime import datetime
time = datetime.now().time().strftime("%Hh_%Mm_%Ss")
date = datetime.now().date().strftime("%Y_%m_%d")
f'{Config.config_path}config_model_{date}_{time}.json'

In [ ]:
f'{Config.config_path}config_model_{version}.json'

In [ ]:
def save_config(config, version):
    import json
    uuid_legth = 36
    fname = f'{config.config_path}config_model_{version[len(version)-uuid_legth:]}.json'
    #fname = f'config_model_{version[len(version)-uuid_legth:]}.json'
    with open(fname, 'w') as file:
        file.write(json.dumps(vars(config))) 

In [ ]:
uuid_legth = 36
f'{config.config_path}config_model_{version[len(version)-uuid_legth:]}.json'

In [ ]:
def get_checkpoint_path(config, version):

    model_name = config.model_name    
    checkpoint_path = config.checkpoint_path
    uuid_legth = 36
    date_legth = 10

    checkpoint_path = f'{Config.checkpoint_path}/{"tibet_gan"}/{version[len(version)-uuid_legth:][:-1]}'

    Path(path).mkdir(parents=True, exist_ok=True)

    return path

In [ ]:
uuid_legth = 36
date_legth = 10
print(checkpoint_path)
#f'{checkpoint_path[:-1]}/{"tibet_gan"}/{version[:date_legth]}/{version[len(version)-uuid_legth:]}'
f'{Config.checkpoint_path}/{"tibet_gan"}/{version[len(version)-uuid_legth:][:-1]}'

In [ ]:
Config.checkpoint_path